# CNN Interpretation

## Load model

In [1]:
from tep.modelUtils import load_architecture

/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model = load_architecture('models/enhanced_meta/baseline.json')

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 32)           0                                            
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 32, 100)      72169600    text_input[0][0]                 
__________________________________________________________________________________________________
pad_1 (ZeroPadding1D)           (None, 34, 100)      0           word_embedding[0][0]             
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 32, 64)       19264       pad_1[0][0]                      
__________________________________________________________________________________________________
pool_1 (Ma

In [4]:
from tep.trainUtils import r2

In [5]:
model.compile(optimizer='Adam', loss='mean_squared_error', metrics=[r2])

In [6]:
model.load_weights('models/enhanced_meta/baseline.hdf5')

## Load data

In [7]:
from tep.utils import load_array
y = load_array("data/log_labels_v3.bc")
print(y.shape)
X_seq = load_array("data/sequences_len32_v3.bc")
print(X_seq.shape)
X_meta = load_array("data/meta_features_v3.bc")
print(X_meta.shape)
emb_mat = load_array("data/embedding_matrix_100dim_v3.bc")
print(emb_mat.shape)

(1293005,)
(1293005, 32)
(1293005, 24)
(721696, 100)


In [23]:
X_valid = X_seq[-10000:]

In [10]:
import json
word_index_file = open('data/word_index_v3.json')
word_index_str = word_index_file.read()
word_index = json.loads(word_index_str)
type(word_index)

dict

## Write analysis functions

In [15]:
import operator

# sort word index for lookups
sorted_word_tuples = sorted(word_index.items(), key=operator.itemgetter(1))
sorted_words = ['<unknown>'] + [t[0] for t in sorted_word_tuples]
sorted_words[:10]

['<unknown>', '.', '<user>', 'to', 'the', '<url>', '!', 'i', '<allcaps>', 'a']

In [16]:
# word lookup function
def lookup_word(word_id):
    return sorted_words[word_id]

In [31]:
import numpy as np

# window lookup function
def get_words(seq, pos, kernel_size):
    pad_size = int(np.floor(np.divide(kernel_size, 2)))
    seq = np.pad(seq, pad_size, 'constant', constant_values=0)
    seq = seq[pos:(pos + kernel_size)]
    words = []
    for word_id in seq:
        word = lookup_word(word_id)
        words.append(word)
    return " ".join(words)

In [32]:
from keras.models import Model

# activation calculator function
def calc_activations(model, layer, batch):
    tmp_model = Model(inputs=model.input[0], outputs=model.get_layer(layer).output)
    activations = tmp_model.predict(batch)
    return activations

In [66]:
# neuron interpreter
def interpret_neuron(model, batch, layer, neuron, num_results=10):
    # get kernel size of conv layer
    kernel_size = model.get_layer(layer).kernel_size[0]
    # calculate layer activations through forward propagation
    activations = calc_activations(model, layer, batch)
    # slice in order to retrieve neuron activations
    neuron_activations = activations[:,:,neuron]
    # sort indices according to hightest activation
    sorted_activations = np.argsort(neuron_activations, axis=None)
    xs, ys = np.unravel_index(sorted_activations, neuron_activations.shape)
    sorted_idx = np.flip(list(zip(xs, ys)), axis=0)[:num_results]
    # assemble results object
    results = []
    for idx in sorted_idx:
        result = {
            'activation': float(neuron_activations[idx[0], idx[1]]),
            'tweet': int(idx[0]),
            'position': int(idx[1]),
            'words': get_words(batch[idx[0]], idx[1], kernel_size),
        }
        results.append(result)
    return results

In [67]:
# layer interpreter
def interpret_layer(model, batch, layer, results_per_neuron=10):
    result_dict = {}
    num_filters = model.get_layer(layer).filters
    for i in range(num_filters):
        result = interpret_neuron(model, batch, layer, i, num_results=results_per_neuron)
        result_dict['Filter_' + str(i)] = result
    return result_dict

In [68]:
interpretations = interpret_layer(model, X_valid, 'conv_1')

In [63]:
!mkdir models/interpretations/

mkdir: cannot create directory ‘models/interpretations/’: File exists


In [70]:
import json

with open('models/interpretations/baseline_conv1.json', 'w') as f:
   json.dump(interpretations, f, indent=4) 